In [1]:
from music21 import *
import os 
import numpy as np
import matplotlib.pyplot as plt
from Utilities.midi_helper import *
import matplotlib.pyplot as plt
import Generate_dataset
import torch
from torch import tensor
import torch.nn as nn
import torch.optim as optim
import tensorflow as tf
import torch.nn.functional as F
import math
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [2]:
#specify the path
path='./schubert/'

#read all the filenames
files=[i for i in os.listdir(path) if i.endswith(".mid")]

#reading each midi file
# notes_array = np.array([read_midi(path+files[0])])
notes_array = np.array([read_midi(path+i) for i in files])

Loading Music File: ./schubert/schub_d760_1.mid
Loading Music File: ./schubert/schumm-1.mid
Loading Music File: ./schubert/schu_143_3.mid
Loading Music File: ./schubert/schuim-4.mid
Loading Music File: ./schubert/schubert_D850_1.mid
Loading Music File: ./schubert/schub_d760_3.mid
Loading Music File: ./schubert/schumm-3.mid
Loading Music File: ./schubert/schumm-5.mid
Loading Music File: ./schubert/schubert_D850_2.mid
Loading Music File: ./schubert/schuim-3.mid
Loading Music File: ./schubert/schub_d960_3.mid
Loading Music File: ./schubert/schub_d960_1.mid
Loading Music File: ./schubert/schubert_D935_3.mid
Loading Music File: ./schubert/schubert_D850_3.mid
Loading Music File: ./schubert/schub_d960_2.mid
Loading Music File: ./schubert/schub_d760_4.mid
Loading Music File: ./schubert/schumm-6.mid
Loading Music File: ./schubert/schumm-2.mid
Loading Music File: ./schubert/schub_d760_2.mid
Loading Music File: ./schubert/schubert_D935_4.mid
Loading Music File: ./schubert/schu_143_1.mid
Loading M

<ipython-input-2-76e837772147>:9: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  notes_array = np.array([read_midi(path+i) for i in files])


In [3]:
x_seq , y_seq , unique_notes,note_to_int  = Generate_dataset.get_sequences(notes_array,32)

/home/god-particle/Desktop/Music_Gen_AI/Generate_dataset.py:21: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  new_music = np.array(new_music)


**Testing the Model by training sinosoids**



*   Uncomment the code chunk below to test o sinosoids 
*   dont run above code if training for sinosoids. Variable names are mostly same



In [4]:
# time = np.linspace(1,10,10000)
# # print(time)
# y = np.sin(20*time)
# # plt.figure();
# # plt.plot(y)
# func = np.round(y,decimals=2)
# # print(y[:100])
# no_of_timesteps = 32
# x = []
# y = []


# for i in range(0, len(func) - no_of_timesteps-1, 1):
    
#     #preparing input and output sequences
#     input_ = func[i:i + no_of_timesteps]
#     output = func[i+no_of_timesteps+1]
    
#     x.append(input_)
#     y.append(output)
        
# x=np.array(x)
# y=np.array(y)

# unique_x = list(set(x.ravel()))
# x_note_to_int = dict((note_, number) for number, note_ in enumerate(unique_x))
# x_seq=[]
# for i in x:
#     temp=[]
#     for j in i:
#         #assigning unique integer to every note
#         temp.append(x_note_to_int[j])
#     x_seq.append(temp)
    
# x_seq = np.array(x_seq)

# unique_y = list(set(y))
# y_note_to_int = dict((note_, number) for number, note_ in enumerate(unique_y)) 
# y_seq=np.array([x_note_to_int[i] for i in y])


In [5]:
from MusicDataset import *

In [6]:
from sklearn.model_selection import train_test_split
x_tr, x_val, y_tr, y_val = train_test_split(x_seq,y_seq,test_size=0.33,random_state=0)

In [7]:
batch_size = 8
train_set = MusicDataset(x_tr,y_tr)
trainloader = torch.utils.data.DataLoader(train_set, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

In [8]:
val_set = MusicDataset(x_val,y_val)
validationloader = torch.utils.data.DataLoader(val_set, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

**DataLoader**

**For Wavenet Model (Ignore for now)**

In [17]:
from Models import Wavenet,LSTM
# Net = Wavenet.Wavenet(len(unique_notes),blocks=4,layers = 2)
Net = LSTM.Sequence(len(unique_notes),32)
Net.to(device)

Sequence(
  (embedding): Embedding(173, 100)
  (lstm): LSTM(100, 256, num_layers=5, batch_first=True)
  (linear1): Linear(in_features=256, out_features=256, bias=True)
  (relu): ReLU()
  (linear2): Linear(in_features=256, out_features=173, bias=True)
  (softmax): Softmax(dim=None)
)

**Checking the model**

In [20]:
import torch.optim as optim
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(Net.parameters(), lr=0.0001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0, amsgrad=False)

**Use the line below when you want to change learning parameters in between**

In [11]:
optimizer = torch.optim.Adam(Net.parameters(), lr=0.001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0, amsgrad=False)

In [12]:
import time , sys
baseDir = '/home/god-particle/Desktop/Music_Gen_AI'
expDir = os.path.join(baseDir, 'trained_model_cache', time.strftime("%d_%m_%Y_%H_%M"))
lossDir = os.path.join(expDir, 'loss')
os.makedirs(expDir, exist_ok=True)
os.makedirs(lossDir, exist_ok=True)
print(baseDir,expDir)

/home/god-particle/Desktop/Music_Gen_AI /home/god-particle/Desktop/Music_Gen_AI/trained_model_cache/30_08_2021_21_26


In [21]:
loss_history = []
best_val_acc = 0

In [ ]:
''' TRAINING THE MODEL '''
correct_preds = 0
total_preds = 0
for epoch in range(20): 
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        input , label = data
        optimizer.zero_grad()
        output = Net(input.to(device),input.shape[0])
#         print('here')
        loss = criterion(output.to(device), label.to(device))
        loss.backward()
        optimizer.step()
        total_preds += input.shape[0]
        correct_preds += torch.sum(torch.argmax(output,1) == label.to(device))

        running_loss += loss.item()
        if (i % 300 == 299  ):
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 300))
            loss_history.append(running_loss/300)
            running_loss = 0.0
    train_acc =  correct_preds/total_preds *100 
    trainreport ="Training Accuracy : \n correct predictions  : {} \n total predictions : {} \n Accuracy : {} \n ------------------------\n".format(correct_preds,total_preds,train_acc)
    print(trainreport)     
    correct_preds = 0
    total_preds = 0
    val_loss = 0
    for i, data in enumerate(validationloader, 0):
        input , label = data
        output = Net(input.to(device),input.shape[0])
        loss = criterion(output.to(device), label.to(device))
        val_loss += loss.item()
        total_preds += input.shape[0]
        correct_preds += torch.sum(torch.argmax(output,1) == label.to(device))
    val_acc = correct_preds/total_preds *100
    val_loss /= total_preds
    valreport ="Validation Accuracy : \n correct predictions  : {} \n total predictions : {} \n Val-Loss : {} \n ------------------------\n".format(correct_preds,total_preds,val_acc )
    print(valreport)
    if(val_acc > best_val_acc):
        print('Updating the saved model')
        best_val_acc = val_acc
        checkpoint = {
            "epoch": epoch,
            "model_state_dict": Net.state_dict(),
            "optimizer_state_dict": optimizer.state_dict(),
            "train_acc": train_acc,
            "valid_acc": val_acc
        }
        torch.save(checkpoint, os.path.join(expDir, 'checkpoint.tar'))
        torch.save(Net, os.path.join(expDir, 'model.pth'))
    
    correct_preds = 0
    total_preds = 0


print('Finished Training')

[1,   300] loss: 4.963
[1,   600] loss: 4.790
[1,   900] loss: 4.732
[1,  1200] loss: 4.664
[1,  1500] loss: 4.618
[1,  1800] loss: 4.611
[1,  2100] loss: 4.611
[1,  2400] loss: 4.547
[1,  2700] loss: 4.537
[1,  3000] loss: 4.533
[1,  3300] loss: 4.559
[1,  3600] loss: 4.492
[1,  3900] loss: 4.499
[1,  4200] loss: 4.507
[1,  4500] loss: 4.482
[1,  4800] loss: 4.471
[1,  5100] loss: 4.468
[1,  5400] loss: 4.432
[1,  5700] loss: 4.440
[1,  6000] loss: 4.414
[1,  6300] loss: 4.412
[1,  6600] loss: 4.391
Training Accuracy : 
 correct predictions  : 1601 
 total predictions : 53827 
 Accuracy : 2.974343776702881 
 ------------------------

Validation Accuracy : 
 correct predictions  : 1220 
 total predictions : 26513 
 Val-Loss : 4.601516246795654 
 ------------------------

Updating the saved model
[2,   300] loss: 4.364
[2,   600] loss: 4.345
[2,   900] loss: 4.352
[2,  1200] loss: 4.323
[2,  1500] loss: 4.284
[2,  1800] loss: 4.302
[2,  2100] loss: 4.293
[2,  2400] loss: 4.261
[2,  2700

In [ ]:
plt.figure()
plt.plot(loss_history)
plt.savefig(lossDir + '/loss_history.png')

In [ ]:
# path='/content/drive/MyDrive/schubert/sine_model.pth'
# path='/content/drive/MyDrive/schubert/lstmmodel1.pth'
path='/content/drive/MyDrive/schubert/sine_wavenet1.pth'
torch.save(Net.state_dict(), path)

In [ ]:
# path='/content/drive/MyDrive/schubert/sine_model.pth'
# path='/content/drive/MyDrive/schubert/lstmmodel1.pth'
checkpoint_path='/home/god-particle/Desktop/Music_Gen_AI/trained_model_cache/29_08_2021_23_57/checkpoint.tar'
Net = LSTM.Sequence(len(unique_notes),32);
checkpoint = torch.load(checkpoint_path)
Net.load_state_dict(checkpoint["model_state_dict"])
print(checkpoint["epoch"])
# Net = Wavenet(len(unique_x),blocks=4,layers = 2)
# Net.load_state_dict(torch.load(path))

correct_preds = 0
total_preds = 0
with torch.no_grad():
  for i, data in enumerate(validationloader, 0):
    input , label = data
    output = Net(input,input.shape[0])
    total_preds += input.shape[0]
    correct_preds += torch.sum(torch.argmax(output,1) == label)
    

report ="correct predictions  : {} \n total predictions : {} \n Accuracy : {}".format(correct_preds,total_preds, correct_preds/total_preds *100)
print(report)
